In [1]:
from os import sys
sys.path.append("/meshAfterParty/")

In [2]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

INFO - 2020-11-30 00:25:31,849 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-30 00:25:31,850 - settings - Setting database.user to celiib
INFO - 2020-11-30 00:25:31,851 - settings - Setting database.password to newceliipass
INFO - 2020-11-30 00:25:31,855 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-30 00:25:31,856 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-30 00:25:31,867 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2020-11-30 00:25:32,059 - settings - Setting enable_python_native_blobs to True


In [3]:
minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-30 00:25:32,078 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-30 00:25:32,269 - settings - Setting enable_python_native_blobs to True


In [ ]:
minnie.AutoProofreadLabels() & "n_errored_synapses<100"

# Function that will visualize neuron with errored synapses

In [ ]:
"""
with certain segment id
1) Pull down the mesh
2) PUll down the synapse data and the error faces
2b) Get the submesh of the errored faces
3) Plot the mesh and the errored faces submesh
"""

segment_id = 864691134884744442 #inhibitory one
segment_id = 864691134884743930 #a false positive (axon thinking was error)
segment_id = 864691134884748026 #good example of caught errors with a little false positives
segment_id = 864691134884745210 # good example of small error detection

#1) Pull down the mesh
current_mesh = du.fetch_segment_id_mesh(segment_id,minnie=minnie)

#2) PUll down the synapse data and the error faces
n_synapses,n_errored_synapses,errored_faces = (minnie.AutoProofreadLabels() &
                                               dict(segment_id=segment_id)).fetch1("n_synapses","n_errored_synapses","face_idx_for_error")

In [ ]:
import neuron_visualizations as nviz
nviz.plot_objects(current_mesh)

In [ ]:
error_submesh = current_mesh.submesh([errored_faces],append=True)
nviz.plot_objects(main_mesh=current_mesh,
                 main_mesh_color=[0,1,0,0.2],
                 meshes=error_submesh,
                  meshes_colors="red",
                 mesh_alpha=1,
                 #html_path="inhibitory_neuron_errored.html",
                 show_at_end=True)

In [ ]:
neuron_obj = (minnie.Decomposition() & dict(segment_id=errored_faces)).fetch1("decomposition")

In [ ]:
import error_detection as ed

In [ ]:
ed.error_faces_by_axons(neuron_obj,verbose=True,
                       non_ais_angle_threshold=60)

In [ ]:
new_error_submeshes

# Visualizing the Actual Synapses that are Errored

In [ ]:
current_mesh

In [ ]:
segment_id

In [ ]:
import error_detection as ed
ed = reload(ed)
err_synapses,non_err_synapses = ed.get_error_synapse_inserts(current_mesh,segment_id,
                                                             errored_faces,return_synapse_centroids=True)

In [ ]:
import trimesh_utils as tu

In [ ]:
valid_mesh = tu.subtract_mesh(current_mesh,error_submesh)

In [ ]:
nviz.plot_objects(main_mesh=valid_mesh,
                  meshes=[error_submesh],
                  meshes_colors=["red"],
                scatters=[err_synapses,non_err_synapses],
                 scatters_colors=["red","yellow"])

# Making suscint function that will plot the cell and the errored synapses

In [13]:
du = reload(du)

In [15]:
du.plot_errored_faces(segment_id=864691134884745210,
                       plot_synapses=False,
                       current_mesh=None,
                       neuron_obj=None,
                       valid_synapse_color = "yellow",
                       error_color = "red")